### Order Analysis: Explore order characteristics, structure, and high-value thresholds.

### 1. Display Order ID and the total amount of each order (each order may include multiple products)

In [ ]:
SELECT OrderID, SUM(OrderedQuantity*ProductStandardPrice) AS Total_Amount
FROM OrderLine_T OLT, Product_T PT      --Commpute Quantity x Price
WHERE OLT.ProductID = PT.ProductID
GROUP BY OrderID;

### 2. List Orders with more than one Product and show number of products in each of these orders

In [ ]:
SELECT OrderID, COUNT(ProductID) AS Num_Product  --Consider OrderID from the OrderLine (Associate Table)
FROM OrderLine_T                                 --however, account for orders that feature multiple products
GROUP BY OrderID
HAVING Num_Product > 1; 

### 3. List total dollar amount for each order that includes ProductID = 2

In [ ]:
SELECT OLT.OrderID, SUM(OrderedQuantity*ProductStandardPrice) AS OrderTotal
FROM OrderLine_T OLT INNER JOIN Product_T PT        --'that includes..' ==> yet is not part of the diplay component
ON OLT.ProductID = PT.ProductID                     --therefore, we implement a subquery to account for orders that 
WHERE OLT.OrderID IN (                              --feature productID 2 (but not only productID 2, that's the diff.)
    OLT.OrderID
    FROM OrderLine_T OLT
    WHERE OLT.ProductID = 2
    )
GROUP BY OLT.OrderID 

### 4. List total dollar amount for each order that includes Coffee Table

In [ ]:
SELECT OrderID, SUM(OrderedQuantity*ProductStandardPrice) AS OrderTotal
FROM OrderLine_T OLT INNER JOIN Product_T PT 
ON OLT.ProductID = PT.ProductID
--'that includes..' ==> yet is not part of the diplay component
-- therefore, we implement a subquery
WHERE OrderID IN (
    OrderID
    FROM OrderLine_T OLT, Product_T PT
    WHERE OLT.ProductID = PT.ProductID
    AND ProductDescription = 'Coffee Table'
    )
GROUP BY OLT.OrderID

### 5. For orders more than $2000, create an invoice and show the total dollar amount of each order. Sort the result by orderID

In [ ]:
SELECT OT.OrderID, OrderDate, CustomerName,
CONCAT(CustomerAddress, " ", CustomerCity, ", ", CustomerState) AS Address,
SUM(OrderedQuantity*ProductStandardPrice) AS OrderTotal
--Joining all four tables
FROM Customer_T CT
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
INNER JOIN Product_T PT ON OLT.ProductID = PT.ProductID
GROUP BY OT.OrderID, OrderDate, CustomerName, Address
HAVING OrderTotal > 2000
ORDER BY OT.OrderID;


### 6. For the most expensive products, display the order ID and customer name

In [ ]:
SELECT OLT.OrderID, CustomerName
FROM Customer_T CT 
INNER JOIN Order_T OT ON CT.CustomerID = OT.CustomerID
INNER JOIN OrderLine_T OLT ON OT.OrderID = OLT.OrderID
--Mult-Subquery allows for bypassing the Product_T join
WHERE OLT.ProductID IN (
	SELECT PT.ProductID							--SQL does not allow us to place Aggregate functions in the WHERE clause
	FROM Product_T PT							--Futhermore, we want to compare EACH record of products to to an MAX price
	WHERE ProductStandardPrice = (				--Once we derive the MAX price 1st, then we can evaluate
		SELECT MAX(ProductStandardPrice)
		FROM Product_T
		)
	)
;